# x402 facilitor for Optimism 

In this notebook, we test the x402 facilitator for Optimism network.

In [9]:
import requests
import json
from datetime import datetime

# Facilitator endpoint (local testing or deployed)
FACILITATOR_URL = "http://localhost:8080"  # Change to deployed URL when available

In [29]:
from eth_account import Account
from eth_account.messages import encode_typed_data
from eth_keys import keys
from web3 import Web3
import os
from dotenv import load_dotenv
import secrets

# Load environment variables
load_dotenv()
private_key = os.getenv('TEST_WALLET_PRIVATE_KEY')
pay_to_address = os.getenv('NFT_WALLET_PUBLIC_KEY')

# Create account from private key
account = Account.from_key(private_key)
from_address = Web3.to_checksum_address(account.address)
pay_to_address = Web3.to_checksum_address(pay_to_address)
print(f"Payer Address: {from_address}")
print(f"Recipient Address: {pay_to_address}")

# Network and token configuration
CHAIN_ID = 11155420  # Optimism Sepolia
USDC_ADDRESS = "0x5fd84259d66Cd46123540766Be93DFE6D43130D7"

# Create authorization data
valid_after = int(datetime.now().timestamp()) - 60
valid_before = int(datetime.now().timestamp()) + 3600
nonce = "0x" + secrets.token_hex(32)
amount = "10000"  # 0.01 USDC (6 decimals)

# EIP-712 domain for USDC on Optimism Sepolia
# IMPORTANT: The contract's name() returns "USDC", not "USD Coin"!
domain_data = {
    "name": "USDC",
    "version": "2",
    "chainId": CHAIN_ID,
    "verifyingContract": USDC_ADDRESS
}

# EIP-712 types for TransferWithAuthorization
types = {
    "EIP712Domain": [
        {"name": "name", "type": "string"},
        {"name": "version", "type": "string"},
        {"name": "chainId", "type": "uint256"},
        {"name": "verifyingContract", "type": "address"}
    ],
    "TransferWithAuthorization": [
        {"name": "from", "type": "address"},
        {"name": "to", "type": "address"},
        {"name": "value", "type": "uint256"},
        {"name": "validAfter", "type": "uint256"},
        {"name": "validBefore", "type": "uint256"},
        {"name": "nonce", "type": "bytes32"}
    ]
}

# Message data
message_data = {
    "from": from_address,
    "to": pay_to_address,
    "value": int(amount),
    "validAfter": int(valid_after),
    "validBefore": int(valid_before),
    "nonce": nonce
}

# Create the full EIP-712 message
full_message = {
    "types": types,
    "primaryType": "TransferWithAuthorization",
    "domain": domain_data,
    "message": message_data
}

# Encode to get the EIP-712 hash components
signable_message = encode_typed_data(full_message=full_message)

# Compute the FULL EIP-712 hash according to spec:
# keccak256("\x19\x01" || domainSeparator || hashStruct(message))
from eth_hash.auto import keccak
prefix = bytes.fromhex("1901")
domain_separator = signable_message.header  # This is hashStruct(domain)
message_hash = signable_message.body  # This is hashStruct(message)
full_eip712_hash = keccak(prefix + domain_separator + message_hash)

# Sign using eth_keys (this creates a pure EIP-712 signature)
private_key_bytes = bytes.fromhex(private_key.replace('0x', ''))
pk = keys.PrivateKey(private_key_bytes)
signature_obj = pk.sign_msg_hash(full_eip712_hash)

# Extract v, r, s components
v_raw, r, s = signature_obj.vrs

# Adjust v to be 27 or 28 (Ethereum standard)
if v_raw == 0:
    v = 27
elif v_raw == 1:
    v = 28
else:
    v = v_raw

# Combine into signature hex string: 0x + r + s + v
r_bytes = r.to_bytes(32, byteorder='big')
s_bytes = s.to_bytes(32, byteorder='big')
signature_hex = '0x' + r_bytes.hex() + s_bytes.hex() + format(v, '02x')

print(f"\nEIP-712 Full Hash (keccak256(\\x19\\x01 || domain || message)): {full_eip712_hash.hex()}")
print(f"\nSignature: {signature_hex}")
print(f"Signature length: {len(signature_hex)} chars (should be 132 with 0x)")
print(f"Nonce: {nonce}")
print(f"Valid After: {valid_after}")
print(f"Valid Before: {valid_before}")
print(f"\n✅ EIP-712 Signatur erstellt")


Payer Address: 0x553179556FC2A39e535D65b921e01fA995E79101
Recipient Address: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

EIP-712 Full Hash (keccak256(\x19\x01 || domain || message)): b15a50aa402614ab92b114dacd2fc170f4a96f45cafbeef5de86402fbdba1efb

Signature: 0x2634380a0fb9c8eadb256632829daf2183d8b46cfe704d2200a0121114b69c3916b66b44309c31b4939babce9bb169a9b8b44434f205bd1e81f12daa87ff47891c
Signature length: 132 chars (should be 132 with 0x)
Nonce: 0x9cb2b28ca8f48cd39b19aa0dcecfd3d733e23db7d52dc823779ede5c51130820
Valid After: 1766001835
Valid Before: 1766005495

✅ EIP-712 Signatur erstellt


### Debug: Prüfe ob Nonce bereits verwendet wurde

Manchmal wurde eine Nonce bereits in einem früheren Test verwendet. Lass uns prüfen:

In [12]:
# Check if the current nonce was already used on-chain
print(f"Prüfe Nonce: {nonce}")
print(f"Payer: {from_address}")

# USDC contract ABI for authorizationState
usdc_abi = [
    {
        "inputs": [
            {"name": "authorizer", "type": "address"},
            {"name": "nonce", "type": "bytes32"}
        ],
        "name": "authorizationState",
        "outputs": [{"name": "", "type": "bool"}],
        "stateMutability": "view",
        "type": "function"
    }
]

# Connect to Optimism Sepolia
w3 = Web3(Web3.HTTPProvider('https://sepolia.optimism.io'))
usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=usdc_abi)

# Check if nonce is used
nonce_used = usdc_contract.functions.authorizationState(from_address, nonce).call()

if nonce_used:
    print(f"\n❌ Diese Nonce wurde bereits verwendet!")
    print(f"Du musst die vorherige Zelle erneut ausführen, um eine neue Nonce zu generieren.")
else:
    print(f"\n✅ Diese Nonce ist noch verfügbar und kann verwendet werden.")

Prüfe Nonce: 0xcfa0a82b51f89222269f47bc29e46dc0b53cb9786e2eaa83f239c1bfc243a5d0
Payer: 0x553179556FC2A39e535D65b921e01fA995E79101

✅ Diese Nonce ist noch verfügbar und kann verwendet werden.

✅ Diese Nonce ist noch verfügbar und kann verwendet werden.


In [30]:
# Now create a valid x402 payment payload with our real signature
real_payment_payload = {
    "x402Version": 2,
    "resource": {
        "url": "https://api.example.com/premium-data",
        "description": "Access to premium market data",
        "mimeType": "application/json"
    },
    "accepted": {
        "scheme": "exact",
        "network": "eip155:11155420",  # Optimism Sepolia testnet
        "amount": amount,
        "asset": USDC_ADDRESS,
        "payTo": pay_to_address,
        "maxTimeoutSeconds": 60,
        "extra": {
            "name": "USDC",
            "version": "2"
        }
    },
    "payload": {
        "signature": signature_hex,
        "authorization": {
            "from": from_address,
            "to": pay_to_address,
            "value": amount,
            "validAfter": str(valid_after),
            "validBefore": str(valid_before),
            "nonce": nonce
        }
    }
}

# Create payment requirements
real_payment_requirements = {
    "scheme": "exact",
    "network": "eip155:11155420",
    "amount": amount,
    "asset": USDC_ADDRESS,
    "payTo": pay_to_address,
    "maxTimeoutSeconds": 60,
    "extra": {
        "name": "USDC",
        "version": "2"
    }
}

# Create verify request
real_verify_request = {
    "paymentPayload": real_payment_payload,
    "paymentRequirements": real_payment_requirements
}

# Send to facilitator
response = requests.post(
    f"{FACILITATOR_URL}/verify",
    headers={"Content-Type": "application/json"},
    json=real_verify_request
)

print(f"Status Code: {response.status_code}")
result = response.json()
print(json.dumps(result, indent=2))

if result.get("isValid"):
    print(f"\n✅ Payment ist GÜLTIG!")
    print(f"Payer: {result.get('payer')}")
else:
    print(f"\n❌ Payment ist UNGÜLTIG")
    print(f"Grund: {result.get('invalidReason')}")

Status Code: 200
{
  "isValid": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101"
}

✅ Payment ist GÜLTIG!
Payer: 0x553179556FC2A39e535D65b921e01fA995E79101


## Settlement via Facilitator

Nach erfolgreicher Verifikation kann der Facilitator die Zahlung on-chain ausführen. Der `/settle` Endpoint:
1. Führt nochmals alle Verifikationen durch
2. Ruft `transferWithAuthorization` auf dem USDC-Contract auf
3. Gibt die Transaction Hash zurück

**Voraussetzungen für Settlement:**
- Der Payer braucht USDC auf Optimism Sepolia
- Der Facilitator braucht einen Private Key (`FACILITATOR_WALLET_PRIVATE_KEY` in `.env`) und etwas ETH für Gas
- Für diesen Test kannst du deinen eigenen Private Key verwenden (im echten Betrieb sollte der Facilitator einen separaten Wallet haben)

### Prüfe Facilitator-Wallet Balance

Bevor wir Settlement versuchen, prüfen wir ob der Facilitator genug ETH für Gas hat:

In [31]:
# Check facilitator wallet balance using public key from .env
facilitator_address = os.getenv('FACILITATOR_WALLET_PUBLIC_KEY')

if facilitator_address:
    facilitator_address = Web3.to_checksum_address(facilitator_address)
    
    # Connect to Optimism Sepolia to check balance
    w3 = Web3(Web3.HTTPProvider('https://sepolia.optimism.io'))
    
    if w3.is_connected():
        balance_wei = w3.eth.get_balance(facilitator_address)
        balance_eth = w3.from_wei(balance_wei, 'ether')
        
        print(f"Facilitator Wallet: {facilitator_address}")
        print(f"ETH Balance: {balance_eth} ETH")
        
        # Estimate gas cost (rough estimate: ~100k gas)
        estimated_gas = 100000
        gas_price = w3.eth.gas_price
        estimated_cost_wei = estimated_gas * gas_price
        estimated_cost_eth = w3.from_wei(estimated_cost_wei, 'ether')
        
        print(f"Geschätzte Gas-Kosten: ~{estimated_cost_eth:.6f} ETH")
        
        if balance_wei == 0:
            print(f"\n❌ FEHLER: Facilitator Wallet hat kein ETH!")
            print(f"Sende ETH an: {facilitator_address}")
            print(f"Bridge: https://app.optimism.io/bridge")
        elif balance_wei < estimated_cost_wei:
            print(f"\n⚠️  WARNUNG: Balance könnte zu niedrig für Gas sein")
            print(f"Empfohlen: Mindestens {estimated_cost_eth:.6f} ETH")
        else:
            print(f"\n✅ Ausreichend ETH für Settlement vorhanden")
    else:
        print("❌ Konnte nicht mit Optimism Sepolia verbinden")
else:
    print("❌ FACILITATOR_WALLET_PUBLIC_KEY nicht in .env konfiguriert")

Facilitator Wallet: 0x3F8d2Fb6fEA24E70155bC61471936F3c9C30c206
ETH Balance: 0.05 ETH
Geschätzte Gas-Kosten: ~0.000000 ETH

✅ Ausreichend ETH für Settlement vorhanden


In [32]:
# Try to settle the payment (execute on-chain)
# Note: This will fail if the payer doesn't have USDC or if the facilitator wallet has no ETH for gas

settle_response = requests.post(
    f"{FACILITATOR_URL}/settle",
    headers={"Content-Type": "application/json"},
    json=real_verify_request  # Same request structure as verify
)

print(f"Status Code: {settle_response.status_code}")
settle_result = settle_response.json()
print(json.dumps(settle_result, indent=2))

if settle_result.get("success"):
    print(f"\n✅ Settlement ERFOLGREICH!")
    print(f"Transaction Hash: {settle_result.get('transaction')}")
    print(f"Network: {settle_result.get('network')}")
    print(f"\nSieh dir die Transaktion an auf:")
    print(f"https://sepolia-optimism.etherscan.io/tx/{settle_result.get('transaction')}")
else:
    print(f"\n❌ Settlement FEHLGESCHLAGEN")
    print(f"Grund: {settle_result.get('errorReason')}")
    if settle_result.get('errorReason') == 'insufficient_funds':
        print(f"\nDer Payer ({settle_result.get('payer')}) hat nicht genug USDC.")
        print(f"Hole dir Test-USDC auf Optimism Sepolia:")
        print(f"1. Bridge von Sepolia ETH → Optimism Sepolia: https://app.optimism.io/bridge")
        print(f"2. Swap ETH → USDC auf Uniswap: https://app.uniswap.org/")

Status Code: 200
{
  "success": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101",
  "transaction": "0x50edaf5d2d743f67218bc4c7f5f5686f074367a8e2db58e0622b148da13b1edb",
  "network": "eip155:11155420"
}

✅ Settlement ERFOLGREICH!
Transaction Hash: 0x50edaf5d2d743f67218bc4c7f5f5686f074367a8e2db58e0622b148da13b1edb
Network: eip155:11155420

Sieh dir die Transaktion an auf:
https://sepolia-optimism.etherscan.io/tx/0x50edaf5d2d743f67218bc4c7f5f5686f074367a8e2db58e0622b148da13b1edb
